In [1]:
from tensorflow.keras.models import model_from_json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import pickle

# load model
with open('rev_sent_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)

# load weights into model
loaded_model.load_weights("rev_sent_model.h5")

# Load tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [2]:
# this function is used for only text
def proces_text_only(text_only):
    # Remove all the special characters (pro_fea = processed feature)
    pro_fea = re.sub(r'\W', ' ', text_only)
    # remove all single characters
    pro_fea = re.sub(r'\s+[a-zA-Z]\s+', ' ', pro_fea)
    # Remove single characters from the start
    pro_fea = re.sub(r'\^[a-zA-Z]\s+', ' ', pro_fea) 
    # Substituting multiple spaces with single space
    pro_fea = re.sub(r'\s+', ' ', pro_fea, flags=re.I)
    # Removing prefixed 'b'
    pro_fea = re.sub(r'^b\s+', '', pro_fea)
    # Converting to Lowercase
    return pro_fea.lower()


stop_words = set(stopwords.words('english'))
    
def prep_sentence(the_text):
    the_text = proces_text_only(the_text)
    word_tokens = word_tokenize(the_text)

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 

    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    the_rev = tokenizer.texts_to_sequences(filtered_sentence)

    ent_list = []

    for sublist in the_rev:
        for item in sublist:
            ent_list.append(item)
            
    return [ent_list]

In [3]:
the_review = "Good place"

instance = pad_sequences(prep_sentence(the_review), padding='post', maxlen=150)

print('This review is positive') if loaded_model.predict(instance)[0][0] > 0.5 else print('This review is negative')
print(loaded_model.predict(instance)[0][0])

This review is positive
0.9778521
